<a href="https://colab.research.google.com/github/chaerui7967/ko_Apartment_Deal/blob/master/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울 집값 예측

In [ ]:
import tensorflow as tf

tf.__version__

'2.6.0'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/project/seoul_5.csv')

In [ ]:
df.head(1)

,gu_l,dong_l,used_y,square,per_cost_man,Price per squar,floor,top10,YN_r,YN_top10,s_c,Hospital,cost
0,0,8,32,77.75,1922.829582,2122.127392,7,0,1,1,1,0.0,149500


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['cost'], axis = 1)
y = df['cost']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

## Standardization

In [ ]:
mean = X_train.mean(axis = 0)
std = X_train.std(axis = 0)

X_train = X_train - mean
X_train = X_train / std

X_test = X_test - mean
X_test = X_test / std

# Train & Validation Split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, 
                                                      test_size = 0.2, 
                                                      random_state = 99) 

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((73050, 12), (18263, 12), (73050,), (18263,))

## DNN
### 1) Model Define

In [ ]:
from tensorflow.keras import models, layers

seoul = models.Sequential(name = 'Regression')
seoul.add(layers.Dense(36, activation = 'relu', input_shape = (12,)))
seoul.add(layers.Dense(60))
seoul.add(layers.BatchNormalization())
seoul.add(layers.Activation('relu'))
seoul.add(layers.Dropout(0.2))
seoul.add(layers.Dense(60))
seoul.add(layers.Activation('relu'))
seoul.add(layers.Dropout(0.2))
seoul.add(layers.Dense(36, activation = 'relu'))
seoul.add(layers.Dense(1))

In [ ]:
seoul.summary()

Model: "Regression"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 36)                468       
_________________________________________________________________
dense_12 (Dense)             (None, 60)                2220      
_________________________________________________________________
batch_normalization_5 (Batch (None, 60)                240       
_________________________________________________________________
activation_5 (Activation)    (None, 60)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 60)                3660      
_________________________________________________________________
activation_6 (Activation)    (None, 60)                0

### 2) Model Compile

In [ ]:
seoul.compile(loss = 'mse',
               optimizer = 'rmsprop',
               metrics = ['mae'])

### 3) Model Fit

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor = 'val_mae', 
                   mode = 'min', 
                   patience = 50, 
                   verbose = 1)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_seoul.h5', 
                     monitor = 'val_mae', 
                     mode = 'min', 
                     save_best_only = True,
                     verbose = 1)

In [ ]:
Hist_seoul = seoul.fit(X_train, y_train,
                         epochs = 500,
                         batch_size = 1,
                         callbacks = [es, mc],
                         validation_data = (X_valid, y_valid))

Epoch 1/500
73050/73050 [==============================] - 203s 3ms/step - loss: 11621308416.0000 - mae: 88718.9219 - val_loss: 11672736768.0000 - val_mae: 88480.3828

Epoch 00001: val_mae improved from inf to 88480.38281, saving model to best_seoul.h5
Epoch 2/500
73050/73050 [==============================] - 205s 3ms/step - loss: 11610346496.0000 - mae: 88659.0938 - val_loss: 11662225408.0000 - val_mae: 88420.2891

Epoch 00002: val_mae improved from 88480.38281 to 88420.28906, saving model to best_seoul.h5
Epoch 3/500
73050/73050 [==============================] - 202s 3ms/step - loss: 11600159744.0000 - mae: 88598.8281 - val_loss: 11651461120.0000 - val_mae: 88359.4844

Epoch 00003: val_mae improved from 88420.28906 to 88359.48438, saving model to best_seoul.h5
Epoch 4/500
73050/73050 [==============================] - 201s 3ms/step - loss: 11589272576.0000 - mae: 88537.5000 - val_loss: 11640719360.0000 - val_mae: 88298.7734

Epoch 00004: val_mae improved from 88359.48438 to 88298.7

### 4) Model Evaluate

In [ ]:
test_mse_score, test_mae_score = boston.evaluate(X_test, y_test)

print('MAE is :',test_mae_score)

### 5) Visualization

In [ ]:
epochs = range(1, len(Hist_seoul.history['val_mae']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, Hist_seoul.history['val_mae'])
plt.title('Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.grid()
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

mae_history = Hist_seoul.history['val_mae']

mae_history = smooth_curve(mae_history[5:])

plt.figure(figsize = (9, 6))
plt.plot(range(1, len(mae_history) + 1), mae_history)
plt.title('Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.grid()
plt.show()